<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/bin_clss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title download
# # https://drive.google.com/file/d/1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q/view?usp=sharing
# !gdown 1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q -O gsv.zip
# !unzip /content/gsv.zip -d /content
# import shutil
# shutil.rmtree('/content/google_street_view/meta_data', ignore_errors=True) # delete the meta_data folder

# # clip cleaned
# # https://drive.google.com/file/d/1-xcHyVAMeTkY7SUdUyQRVAn_FFXPSDGB/view?usp=share_link
!gdown 1-xcHyVAMeTkY7SUdUyQRVAn_FFXPSDGB -O gsv.zip
!unzip /content/gsv.zip -d /
!rm -R /content/gsv/.ipynb_checkpoints
!rm -R /content/gsv/01/.ipynb_checkpoints
!rm -R /content/gsv/02/.ipynb_checkpoints
!rm -R /content/gsv/03/.ipynb_checkpoints
!rm -R /content/gsv/04/.ipynb_checkpoints
!rm -R /content/gsv/05/.ipynb_checkpoints
!rm -R /content/gsv/06/.ipynb_checkpoints

# # gsv 70k
# # https://drive.google.com/file/d/1-7ZC29k4VxXQkpnOuLfj7Ag_SFTM4LV8/view?usp=share_link
# !gdown 1-7ZC29k4VxXQkpnOuLfj7Ag_SFTM4LV8 -O gsv.zip
# !unzip /content/gsv.zip -d /

# # # !ls
# !ls -a /content/gsv70k
# !rm -R /content/gsv70k/.ipynb_checkpoints
# # # !rm -R /content/gsv/06/.ipynb_checkpoints

# # https://bestasoff.medium.com/how-to-fine-tune-very-large-model-if-it-doesnt-fit-on-your-gpu-3561e50859af
!pip install bitsandbytes


In [ ]:
# @title augmentations
# https://github.com/facebookresearch/vicreg/blob/main/augmentations.py

from PIL import ImageOps, ImageFilter
import numpy as np
import torchvision.transforms as transforms
from torchvision.transforms import InterpolationMode

class GaussianBlur(object):
    def __init__(self, p):
        self.p = p
    def __call__(self, img):
        if np.random.rand() < self.p:
            sigma = np.random.rand() * 1.9 + 0.1
            # return img.filter(ImageFilter.GaussianBlur(sigma))
            return transforms.GaussianBlur(kernel_size=5, sigma=sigma)(img)
        else:
            return img

class Solarization(object):
    def __init__(self, p):
        self.p = p
    def __call__(self, img):
        if np.random.rand() < self.p:
            return ImageOps.solarize(img)
        else:
            return img

class Cutout(object):
    def __init__(self, p=0.5, mask_size=140, mask_color=(0, 0, 0)):
        self.p = p
        self.mask_size = mask_size
        self.mask_color = mask_color
        self.mask_size_half = self.mask_size // 2
        # offset = 1 if mask_size % 2 == 0 else 0
    def __call__(self, img):
        if np.random.rand() > self.p:
            return img
        else:
            img = np.asarray(img).copy()
            h, w = img.shape[:2] # 480x600
            cx = np.random.randint(0, w)
            cy = np.random.randint(0, h)
            xmin, xmax = cx - self.mask_size_half, cx + self.mask_size_half
            ymin, ymax = cy - self.mask_size_half, cy + self.mask_size_half
            xmin, xmax = max(0, xmin), min(w, xmax)
            ymin, ymax = max(0, ymin), min(h, ymax)
            img[ymin:ymax, xmin:xmax] = self.mask_color
            return img


class TrainTransform(object):
    def __init__(self):
        self.transform = transforms.Compose([
                transforms.RandomPerspective(distortion_scale=0.3, p=0.5), # me
                # transforms.RandomResizedCrop(224, interpolation=InterpolationMode.BICUBIC),#224
                # transforms.RandomResizedCrop((400,640), interpolation=InterpolationMode.BICUBIC),#224
                # transforms.RandomResizedCrop((400,640), scale=(0.8, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=InterpolationMode.BICUBIC),
                transforms.RandomResizedCrop((400,640), scale=(0.7, 1.0), ratio=(0.8, 1.25), interpolation=InterpolationMode.BICUBIC),
                transforms.RandomHorizontalFlip(p=0.5), # 0.5
                transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8,), # brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8
                transforms.RandomGrayscale(p=0.2), # 0.2
                GaussianBlur(p=1.0), # 1.0
                # Solarization(p=0.0), # 0.0
                Cutout(p=1.0),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225
            ])

    def __call__(self, sample):
        x1 = self.transform(sample)
        # x2 = transforms.ToTensor()(sample)
        return x1
# https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py



# https://github.com/hysts/pytorch_cutout/blob/master/dataloader.py
# https://arxiv.org/pdf/1708.04552.pdf
# size of the cutout region is a more important hyperparameter than the shape
# randomly select a pixel coordinate within the image as a center point and then place the cutout mask around that location.
# alternative randomly apply cutout constrained within the image region, but with 50% probability so that the network sometimes receives unmodified images
def cutout(p=0.5, mask_size=16, mask_color=(0, 0, 0)):
    mask_size_half = mask_size // 2
    # offset = 1 if mask_size % 2 == 0 else 0
    def _cutout(image):
        image = np.asarray(image).copy()
        if np.random.random() > p:
            return image
        h, w = image.shape[:2]
        cx = np.random.randint(0, w)
        cy = np.random.randint(0, h)
        xmin, xmax = cx - mask_size_half, cx + mask_size_half
        ymin, ymax = cy - mask_size_half, cy + mask_size_half
        xmin, xmax = max(0, xmin), min(w, xmax)
        ymin, ymax = max(0, ymin), min(h, ymax)
        image[ymin:ymax, xmin:xmax] = mask_color
        return image
    return _cutout



In [2]:
# @title data weighted
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# dataset has PILImage images of range [0, 1], transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
# transform = transforms.Compose(transforms.ToTensor())

# dir='/content/google_street_view'
# dir='/content/gsv70k'
dir='/content/gsv'

# data = datasets.ImageFolder(dir, transform=transform)
data = datasets.ImageFolder(dir, transform=None)
torch.manual_seed(0)

# train_data, test_data = torch.utils.data.random_split(data, [.85,.15])
# https://www.scaler.com/topics/pytorch/how-to-split-a-torch-dataset/
data_size = len(data)
indices = np.arange(data_size)
np.random.shuffle(indices)
split_index = int(np.floor(0.9 * data_size))
train_idx, test_idx = indices[:split_index], indices[split_index:]
train_data = torch.utils.data.Subset(data, train_idx)
test_data = torch.utils.data.Subset(data, test_idx)
targets = np.array(data.targets)
train_targets = targets[train_idx]
test_targets = targets[test_idx]


from torch.utils.data import Dataset
class DatasetWrap(Dataset):
    def __init__(self, dataset, transform=None):
        super(DatasetWrap, self).__init__()
        self.dataset = dataset
        self.transform = transform
        self.datasize = len(self.dataset)

    def __len__(self):
        return self.datasize
        
    # def __getitem__(self, index):
    #     x, y = self.dataset[index]
    #     x1, y1 = self.dataset[(index+1)%self.datasize]
    #     if y>y1: yy=0.
    #     elif y<y1: yy=1.
    #     elif y==y1: yy=0.5
    #     if self.transform:
    #         x = self.transform(x)
    #         x1 = self.transform(x1)
    #     return (x,x1), yy
    def __getitem__(self, index):
        x, y = self.dataset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

# train_data = DatasetWrap(train_data, TrainTransform()) # apply data augmentation to train dataset only
train_data = DatasetWrap(train_data, transform)
test_data = DatasetWrap(test_data, transform)

batch_size = 16 # 64

# https://stackoverflow.com/questions/62319228/number-of-instances-per-class-in-pytorch-dataset
from collections import Counter
class_count = torch.tensor(list(Counter(data.targets).values()))
weights=1./class_count
# weights=sum(class_count)/class_count
# print(weights)
# https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler

train_weight = weights[train_targets]
test_weight = weights[test_targets]
train_sampler = torch.utils.data.WeightedRandomSampler(train_weight, len(train_weight))
test_sampler = torch.utils.data.WeightedRandomSampler(test_weight, len(test_weight))
train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
data, train_data, test_data = None, None, None

num_batches=int(np.ceil(len(train_loader)/batch_size))

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# dataiter = iter(train_loader)
# images, labels = next(dataiter)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# dataiter=None
# print(labels)

# dataiter = iter(test_loader)
# images, labels = next(dataiter)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# dataiter=None


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [50]:

dir='/content/gsv'

combi = torch.combinations(torch.arange(0,6), with_replacement=True)
i, j = 1,2

datai = datasets.ImageFolder(dir+'/'+str(i).zfill(2), transform=None)
dataj = datasets.ImageFolder(dir+'/'+str(j).zfill(2), transform=None)
train_bin_set = torch.utils.data.ConcatDataset([datai, dataj])






FileNotFoundError: ignored

In [27]:
# @title resnet
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
# # https://pytorch.org/vision/0.12/models.html#id10
# model = models.resnet18(pretrained=True) # 18 34 50 101 152
model = models.resnet18(weights='DEFAULT') # 18 34 50 101 152
num_ftrs = model.fc.in_features
model.fc = nn.Identity()
bincls = nn.Sequential(nn.Linear(num_ftrs*2, num_ftrs), nn.ReLU(), # for bin classifier
                nn.Linear(num_ftrs, 1), nn.Sigmoid(),)
# bincls = nn.Linear(num_ftrs*2, 1) # for bin classifier


# # model.mods = [module for k, module in model._modules.items()]
# # modules = [module for k, module in model._modules.items()]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
bincls = bincls.to(device)
# model = torch.compile(model.to(device))
# model = torch.compile(model.to(device),mode='max-autotune')
# bincls = torch.compile(bincls.to(device),mode='max-autotune')


In [ ]:
# @title try

# # check model's input and output dimensions are correct
x0 = torch.rand(16, 3, 400, 640, device=device)
x1 = torch.rand(16, 3, 400, 640, device=device)
model.eval()
pred0 = model(x0)
pred1 = model(x1)
print(pred0.shape)
print(torch.cat((pred0, pred1),).shape)

pred = bincls(torch.cat((pred0, pred1),-1))

# targets = torch.randint(0,6,(16,), device=device)
targets = torch.randint(0,3,(16,), device=device)/2
print(targets)



In [39]:
# @title train test function
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()
# https://github.com/prigoyal/pytorch_memonger/blob/master/models/optimized/resnet_new.py
from torch.utils.checkpoint import checkpoint, checkpoint_sequential

def triangle(bincls,pred,y,loss_fn):
    combi = torch.combinations(torch.arange(0,y.shape[0]), with_replacement=True)
    logits = bincls(torch.cat((pred[combi[:,0],:], pred[combi[:,1],:]),-1)).squeeze()
    # yy = torch.where(y[combi[:,0]]>y[combi[:,1]], 0., torch.where(y[combi[:,0]]<y[combi[:,1]], 1., 0.5))
    yy = torch.where(y[combi[:,0]]>y[combi[:,1]], 0., 1.)
    loss = loss_fn(logits, yy)
    return loss

# train function with automatic mixed precision
def strain(dataloader, model, loss_fn, optimizer, scheduler=None, verbose=True):
    model,bincls=model
    size = len(dataloader.dataset)
    model.train()
    loss_list = []
    for batch, (x, y) in enumerate(dataloader):
        sx, sy = x.to(device), y.to(device)
        # sy=sy.unsqueeze(-1)
        with torch.cuda.amp.autocast(): # automatic mixed percision
            pred = model(sx) # [16, 512]
            # loss = loss_fn(pred, sy)
            loss=triangle(bincls,pred,sy,loss_fn)
            # modules = [module for k, module in model._modules.items()]
            # pred = checkpoint_sequential(functions=modules, segments=1, input=sx) # gradient checkpointing for resnet and inception only
            # # # pred = checkpoint_sequential(functions=model.mods, segments=1, input=sx)
            # loss = loss_fn(pred, sy) # /4 to scale by gradient accumulation_steps

        scaler.scale(loss).backward()
        # if ((batch + 1) % 4 == 0) or (batch + 1 == len(dataloader)): # gradient accumulation
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        if scheduler is not None:
            scheduler.step()

        loss_list.append(loss.item())
        # loss.backward()
        # optimizer.step()
        if (batch) % (size//(10* len(x))) == 0:
            loss, current = loss.item(), batch * len(x)
            if verbose: print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    return loss_list

def test(dataloader, model, loss_fn, verbose=True):
    model,bincls=model
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (x, y) in enumerate(dataloader):
            sx, sy = x.to(device), y.to(device)
            pred = model(sx)
            # loss = loss_fn(pred, y)
            # # # predicted, actual = classes[pred[0].argmax(0)], classes[y]

            # test_loss += loss_fn(pred, y).item()
            test_loss += triangle(bincls,pred,y,loss_fn).item()
            # correct += ((pred*2).round() == y).type(torch.float).sum().item()

    test_loss /= num_batches
    # correct /= size
    if verbose: print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    # if verbose: print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    # return correct, test_loss
    return test_loss


In [ ]:
pth='/content/lr.pth'
torch.save(model.state_dict(), pth) # save temporary model for lr finding


In [ ]:
# @title LR range test
# 1cycle super convergencehttps://arxiv.org/pdf/1708.07120.pdf
# # cyclic lr https://arxiv.org/pdf/1506.01186.pdf
# Note the learning rate value when the accuracy starts to
# increase and when the accuracy slows, becomes ragged, or starts to fall

# one training run of the network for a few epochs

epochs=5
min_lr= 1e-7
max_lr= 1e-3 # 1e-2
# 152: 1e-7 - 1e-4      result 3e-7 - 3e-6

# loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.BCEWithLogitsLoss() # loss with sigmoid, so model no need sigmoid

model.load_state_dict(torch.load("lr.pth"))
# optimizer = torch.optim.AdamW(model.parameters(), lr=start_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
# optimizer = torch.optim.SGD(model.parameters(), lr=start_lr, momentum=0.9)
import bitsandbytes as bnb
# optimizer = bnb.optim.AdamW(model.parameters(), lr=min_lr, betas=(0.9, 0.999), optim_bits=8)
optimizer = bnb.optim.AdamW(list(model.parameters())+ list(bincls.parameters()), lr=min_lr, betas=(0.9, 0.999), optim_bits=8)

# num_batches=int(np.ceil(len(test_loader)/batch_size))
num_batches=len(test_loader)

# total_steps=int(np.ceil(num_batches/4)*epochs)
total_steps=int(num_batches*epochs)

# min_lr* gamma^total_steps = max_lr
gamma = np.exp(np.log(max_lr/min_lr)/total_steps) # for scheduler step every optimizer step
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # 0.75(20)-0.9(100)
train_lst, test_lst=[],[]
lr_list=np.ones(total_steps)*min_lr*gamma**np.arange(total_steps)
# gamma = np.exp(np.log(max_lr/min_lr)/(total_steps*4)) # total_steps*4 bec grad accumulation, loss step 4x per lr step
# lr_list=np.ones(total_steps*4)*min_lr*gamma**np.arange(total_steps*4)


for i in range(epochs):
    train_ls = strain(test_loader, model, loss_fn, optimizer, scheduler)    
    train_lst.extend(train_ls)


# https://stackoverflow.com/a/53472966/13359815
from scipy.ndimage.filters import gaussian_filter1d
train_lstsm = gaussian_filter1d(train_lst, sigma=3)
end=200

import matplotlib.pyplot as plt
plt.plot(lr_list, train_lst)
plt.plot(lr_list, train_lstsm)
# plt.plot(lr_list[:end], train_lst[:end])
# plt.plot(lr_list[:end], train_lstsm[:end])
plt.xscale('log')
# plt.plot(train_lst)
# plt.plot(np.linspace(0,len(train_lst),len(test_lst)), test_lst)
plt.show()
plt.close()


In [ ]:
print(len(lr_list))
start, end = 20, 220
plt.plot(lr_list[:end], train_lst[:end])
plt.plot(lr_list[:end], train_lstsm[:end])
plt.xscale('log')
plt.show()

In [40]:
# @title wwwwwwwww
import time
start = time.time()
acc_lst, train_lst, test_lst=[],[],[]
# loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.BCELoss()
# loss_fn = nn.BCEWithLogitsLoss() # loss with sigmoid, so model no need sigmoid
loss_fn = nn.L1Loss()
# https://pytorch.org/docs/stable/nn.html#loss-functions

base_lr, max_lr = 1e-6, 3e-5 # 1e-5, 1e-3 #0.5#
# res152 3e-7, 3e-6
# base_lr, max_lr = 3e-6, 3e-4
end_lr, start_lr = 1e-5, 1e-3 # 0.0001,0.1

epochs = 5 #5 20
# optimizer = torch.optim.SGD(model.parameters(), lr = start_lr, momentum=0.9)
# optimizer = torch.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)
import bitsandbytes as bnb # 8bit optimizer
# optimizer = bnb.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), optim_bits=8)
optimizer = bnb.optim.AdamW(list(model.parameters())+ list(bincls.parameters()), lr=base_lr, betas=(0.9, 0.999), optim_bits=8)
# optimizer = bnb.optim.AdamW(model.parameters(), lr=1e-5, betas=(0.9, 0.999), optim_bits=8)
# 152 1e-5
# cnn 3e-4

div_factor = max_lr/base_lr
num_batches=len(train_loader)
total_steps=int(num_batches*epochs)+1
# total_steps=int(np.ceil(num_batches/4)*epochs)+1 # +1 to excluse uptick at the end of onecycle
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=100.0, three_phase=True,)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, epochs=epochs, steps_per_epoch=num_batches, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=10000.0, three_phase=True,)
# gamma = np.exp(np.log(end_lr/start_lr)/(num_batches*epochs)) # for scheduler step every optimizer step
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # 0.75(20)-0.9(100)


# for x in range(int(num_batches)*1):
#     scheduler.step()


for t in range(0,epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    lr=optimizer.param_groups[0]["lr"]
    print(lr)
    # train_ls = strain(train_loader, model, loss_fn, optimizer, scheduler)
    # correct, test_loss = test(test_loader, model, loss_fn)
    train_ls = strain(train_loader, (model,bincls), loss_fn, optimizer, scheduler)
    # correct, test_loss = test(test_loader, (model,bincls), loss_fn)
    train_lst.extend(train_ls)
    # test_lst.append(test_loss)
    # acc_lst.append(correct)

    # checkpoint = { # https://discuss.pytorch.org/t/saving-model-and-optimiser-and-scheduler/52030
    # 'epoch': t,
    # 'model': model.state_dict(),
    # 'optimizer': optimizer.state_dict(),
    # 'lr_sched': scheduler}
    # # torch.save(checkpoint, 'checkpoint.pth')
    # torch.save(checkpoint, '/content/drive/MyDrive/frame/resnet1522.pth')
    # torch.save(model.state_dict(), "model.pth")

print("Done!")

end = time.time()
print("time: ",end - start)

# print(len(train_lst), len(test_lst))
import matplotlib.pyplot as plt
plt.plot(train_lst)
# plt.plot(np.linspace(0,len(train_lst),len(test_lst)), test_lst.cpu().numpy())
# plt.plot(np.linspace(0,len(train_lst),len(test_lst)), test_lst)
plt.show()
plt.plot(acc_lst)
plt.show()
plt.close()


torch.save(model.state_dict(), "model.pth")

# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

# 38.9%, Avg loss: 0.725188  946s



Epoch 1
-------------------------------
1.0000000000000006e-06
torch.Size([16, 512])
loss: 0.204686 [    0/ 7421]
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16, 512])


Exception in thread Thread-36 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 51, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 307, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3

KeyboardInterrupt: ignored

In [ ]:
# test_loss = test(test_loader, (model,bincls), loss_fn)
correct, test_loss = test(test_loader, (model,bincls), loss_fn)


Test Error: 
 Accuracy: 15.8%, Avg loss: 0.924583 



In [36]:
# @title get know preds
import os
from PIL import Image
import matplotlib.pyplot as plt

img_dir = '/content/gsv/'

size=3
pred_lst=torch.empty((6,size,512),device=device)
# get know preds
from torchvision import transforms
for x in range(6):
    cls=x+1
    filelist = np.array(os.listdir(img_dir+str(cls).zfill(2)))
    clsfiles =filelist[np.random.randint(0, len(filelist), (size))]
    for i, filename in enumerate(clsfiles):
        img_file=os.path.join(img_dir, str(cls).zfill(2)+'/'+filename)
        image = Image.open(img_file).convert("RGB")
        knw=transforms.ToTensor()(image).to(device).unsqueeze(0)
        pred = model(knw)
        # print(pred.shape) # [1, 512]
        pred_lst[x][i]=pred
print(pred_lst.shape) # [6, 3, 512]
print(pred_lst)

torch.Size([6, 3, 512])
tensor([[[0.6953, 0.7724, 0.7501,  ..., 0.6733, 0.8651, 0.7300],
         [0.6249, 0.7091, 0.7422,  ..., 0.6241, 0.7994, 0.6429],
         [0.7246, 0.7467, 0.8165,  ..., 0.7507, 0.9288, 0.7241]],

        [[0.6624, 0.6780, 0.7411,  ..., 0.6460, 0.8356, 0.6810],
         [0.6479, 0.6958, 0.7414,  ..., 0.6546, 0.8542, 0.6489],
         [0.5345, 0.5481, 0.6609,  ..., 0.5365, 0.7257, 0.5516]],

        [[0.7189, 0.7468, 0.8103,  ..., 0.6916, 0.9092, 0.7441],
         [0.6613, 0.7347, 0.7540,  ..., 0.6456, 0.8192, 0.7092],
         [0.5204, 0.5337, 0.6496,  ..., 0.5333, 0.7200, 0.5258]],

        [[0.5814, 0.6279, 0.6894,  ..., 0.5872, 0.7957, 0.6053],
         [0.7469, 0.6885, 0.7818,  ..., 0.7023, 0.9330, 0.7751],
         [0.8003, 0.7500, 0.8270,  ..., 0.7400, 0.9562, 0.8073]],

        [[0.5369, 0.5235, 0.6358,  ..., 0.5235, 0.7166, 0.5707],
         [0.6481, 0.6314, 0.7550,  ..., 0.6373, 0.8746, 0.6697],
         [0.6392, 0.6140, 0.7433,  ..., 0.6253, 0.8434, 0.

In [35]:
img,label=next(iter(test_loader))
# print(img.shape) # [16, 3, 400, 640]
img=img.to(device)
pimg = model(img)
# print(pimg.shape) # [16, 512]
# dpimg=pimg.expand(size,-1,-1)
dpimg=pimg.unsqueeze(1).expand(-1,size,-1) # [16, 3, 512]
# dpimg=pimg.expand(-1,size,512)
# print(dpimg.shape) # [16, 3, 512] batch,size,hidden
print(dpimg)


# clsls=[]
clsls=torch.empty(batch_size,6)
# clsvote=torch.empty(batch_size)
for x, dlst in enumerate(pred_lst): # kwn per cls
    cls=x+1
    # yy = torch.where(y[combi[:,0]]>y[combi[:,1]], 0., torch.where(y[combi[:,0]]<y[combi[:,1]], 1., 0.5))
    # print(dlst.shape) # [3, 512]
    dlst=dlst.unsqueeze(0).expand(batch_size,-1,-1) # [16, 3, 512]
    pred = bincls(torch.cat((dlst, dpimg),-1)).squeeze() # [16, 3, 1]
    # loss = loss_fn(pred, y)
    # print(pred.shape, label.shape) # [16, 3], [16]
    # clsls[x]=loss_fn(pred, label).sum()
    clsls[:,x]=pred.sum(-1) # [16]
    # # # predicted, actual = classes[pred[0].argmax(0)], classes[y]
    # test_loss += loss_fn(pred, y).item()
    # correct += ((pred*2).round() == y).type(torch.float).sum().item()

torch.set_printoptions(precision=4)
# torch.set_printoptions(profile='default')
# print(label)
print(clsls/size)
print(clsls.shape)


tensor([[[0.2047, 0.3870, 0.2975,  ..., 0.2427, 0.2582, 0.2955],
         [0.2047, 0.3870, 0.2975,  ..., 0.2427, 0.2582, 0.2955],
         [0.2047, 0.3870, 0.2975,  ..., 0.2427, 0.2582, 0.2955]],

        [[1.2774, 1.3094, 1.3688,  ..., 1.3201, 1.6761, 1.2368],
         [1.2774, 1.3094, 1.3688,  ..., 1.3201, 1.6761, 1.2368],
         [1.2774, 1.3094, 1.3688,  ..., 1.3201, 1.6761, 1.2368]],

        [[0.1924, 0.4473, 0.3157,  ..., 0.2722, 0.2568, 0.2669],
         [0.1924, 0.4473, 0.3157,  ..., 0.2722, 0.2568, 0.2669],
         [0.1924, 0.4473, 0.3157,  ..., 0.2722, 0.2568, 0.2669]],

        ...,

        [[0.3807, 0.5357, 0.5102,  ..., 0.4150, 0.5611, 0.4218],
         [0.3807, 0.5357, 0.5102,  ..., 0.4150, 0.5611, 0.4218],
         [0.3807, 0.5357, 0.5102,  ..., 0.4150, 0.5611, 0.4218]],

        [[0.2838, 0.4154, 0.2905,  ..., 0.3016, 0.3218, 0.3429],
         [0.2838, 0.4154, 0.2905,  ..., 0.3016, 0.3218, 0.3429],
         [0.2838, 0.4154, 0.2905,  ..., 0.3016, 0.3218, 0.3429]],

 

In [49]:
test_iter = iter(test_loader)
img,label=next(test_iter)
img=img.to(device)

# img[0]
pimg0 = model(img[0].unsqueeze(0))
pimg1 = model(img[1].unsqueeze(0))
pred = bincls(torch.cat((pimg0, pimg1),-1)).squeeze() # [16, 3, 1]
print(pred)
print(label[:2])




tensor(0.8345, device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([5, 3])


In [37]:
# @title confusion matrix
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns

model.eval()
y_true, y_pred = [], []
img,label=next(iter(test_loader))
img=
with torch.no_grad():
    # print(img.shape) # [16, 3, 400, 640]
    img=img.to(device)
    pimg = model(img)
    for (X,X1), y in test_loader:
        x0, x1, y = X.to(device), X1.to(device), y.to(device)
        pred0 = model(x0)
        pred1 = model(x1)
        pred = bincls(torch.cat((pred0, pred1),-1)).squeeze()
        predicted_labels = (pred*2).round()
        print(y,predicted_labels)
        y_true.extend(list(y.cpu()))
        y_pred.extend(list(predicted_labels.cpu()))



def triangle(bincls,pred,y,loss_fn):
    combi = torch.combinations(torch.arange(0,y.shape[0]), with_replacement=True)
    logits = bincls(torch.cat((pred[combi[:,0],:], pred[combi[:,1],:]),-1)).squeeze()
    # yy = torch.where(y[combi[:,0]]>y[combi[:,1]], 0., torch.where(y[combi[:,0]]<y[combi[:,1]], 1., 0.5))
    yy = torch.where(y[combi[:,0]]>y[combi[:,1]], 0., 1.)
    loss = loss_fn(logits, yy)
    return loss


cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=['1','2','3','4','5','6',], yticklabels=['1','2','3','4','5','6',])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


ValueError: ignored

In [ ]:
# @title save

# from google.colab import drive
# drive.mount('/content/drive')

# # pth='/content/drive/MyDrive/frame/vit.pth'
# pth='/content/drive/MyDrive/frame/res55.pth'
# pth='/content/drive/MyDrive/frame/resnet152.pth'

torch.save(model.state_dict(), pth)
# model.load_state_dict(torch.load(pth))
# # model.load_state_dict(torch.load("model.pth"))





# # https://drive.google.com/file/d/1visTNvWmnuV7jAm2TBiAIIrNjbOAi1Fv/view?usp=share_link
# !gdown 1visTNvWmnuV7jAm2TBiAIIrNjbOAi1Fv -O resnet152.pth

# # t, modelsd, optimsd, scheduler = torch.load('/content/drive/MyDrive/frame/resnet152.pth').values()
# t, modelsd, optimsd, scheduler = torch.load('/content/resnet152.pth').values()
# model.load_state_dict(modelsd)
# # optimizer.load_state_dict(optimsd)



# # matt152 # https://drive.google.com/file/d/1MQ0xLfHbio458uEVbn2VyMpD3bij2A4J/view?usp=sharing
# !gdown 1MQ0xLfHbio458uEVbn2VyMpD3bij2A4J -O res152.pth
# model.load_state_dict(torch.load("res152.pth"))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
